# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,woolgoolga,-30.1106,153.2007,85.95,71,0,19.35,AU,1732586841
1,1,copala,16.6500,-98.9833,77.02,84,10,3.00,MX,1732586937
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,36.19,48,75,11.18,RU,1732586938
3,3,dryden,49.7833,-92.7503,21.49,86,100,11.50,CA,1732586939
4,4,n'zeto,-7.2312,12.8666,78.08,85,100,7.11,AO,1732586941


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
humidityMap = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo =True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha =.7,
    tiles = "OSM",
    frame_height = 500,
    frame_width = 800,
    hover_cols = ["City", "Humidity"]
)  

# Display the map
humidityMap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness

IdealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] <80.6)\
    & (city_data_df["Wind Speed"] <4.5)\
    & (city_data_df["Cloudiness"] ==0)
]

# Drop any rows with null values
IdealCityDF = IdealCityDF.dropna()

# Display sample data
print(len(IdealCityDF))
IdealCityDF.head()

4


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,ixtapa,20.7000,-105.2000,76.23,83,0,0.00,MX,1732586945
159,159,lazaro cardenas,17.9583,-102.2000,78.01,77,0,3.56,MX,1732587092
175,175,jaru,-10.4389,-62.4664,73.09,96,0,1.70,BR,1732587145
304,304,salalah,17.0151,54.0924,75.29,69,0,2.30,OM,1732587296


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = IdealCityDF[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,ixtapa,MX,20.7000,-105.2000,83,
159,lazaro cardenas,MX,17.9583,-102.2000,77,
175,jaru,BR,-10.4389,-62.4664,96,
304,salalah,OM,17.0151,54.0924,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
# filter out where there are no hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "no hotel found"]
hotel_df

Starting hotel search
ixtapa - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
jaru - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,ixtapa,MX,20.7000,-105.2000,83,No hotel found
159,lazaro cardenas,MX,17.9583,-102.2000,77,No hotel found
175,jaru,BR,-10.4389,-62.4664,96,No hotel found
304,salalah,OM,17.0151,54.0924,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
##%%capture --no-display

# Configure the map plot
HotelCountry = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .7,
    tiles = "OSM",
    frame_height = 500,
    frame_width = 800,
    hover_cols = ["City","Hotel Name", "Humidity", "Country"]
)  
HotelCountry

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)